In [1]:
import io
import requests
import tabula
headers = {'User-Agent': 'Mozilla/5.0 (X11; Windows; Windows x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36'}

url = 'https://www.digihealthcare.scot/app/uploads/2024/05/CDI-Standard-V4.5-FINAL.pdf'
response = requests.get(url=url, headers=headers, timeout=120)
on_fly_mem_obj = io.BytesIO(response.content)
dfs = tabula.read_pdf(on_fly_mem_obj, pages="12-27", stream=True, encoding='utf-8', pandas_options={'header': None}, lattice=True)

Failed to import jpype dependencies. Fallback to subprocess.
No module named 'jpype'


In [2]:
print(len(dfs))

117


In [3]:
dfs[7]

,0,1,2,3,4,5
0,REVISED DOCUMENT INDEXING STANDARDS (May 2024),NaN,NaN,NaN,NaN,NaN
1,REVISED DOCUMENT INDEXING STANDARDS (May 2024),NaN,NaN,NaN,NaN,NaN
2,DST Code,Document\rType/Subtype,NaN,escription,NaN,SNOMED Code
3,NaN,ocument,NaN,NaN,NaN,NaN
4,ST Code,examples where,SNOMED Code,NaN,NaN,NaN
5,NaN,ype/Subtype,NaN,NaN,NaN,NaN
6,NaN,NaN,pplicable),NaN,NaN,NaN
7,NaN,NaN,specialist involvement\rwhere appropriate.,NaN,NaN,NaN
8,AS05,CPA assessment,Care Programme\rApproach.,820031000000102\rCare Programme Approach\rAsse...,NaN,NaN
9,AS07,Multidisciplinary\rassessment,Any assessment\rcompleted by various\rclinical...,820041000000106\rMultidisciplinary\rAssessment...,NaN,NaN


In [4]:
from fhirpy import SyncFHIRClient
import requests
import json

client = SyncFHIRClient(url='https://tx.fhir.org/r4', extra_headers={})
for df in dfs:

    if (len(df.columns)>1):
        df[3] = df[3].astype(str)
 #       print(df.dtypes)
        for index, row in df.iterrows():
            if len(row[3])>5 and row[3].isnumeric():
                try:
                    request = 'https://tx.fhir.org/r4/CodeSystem/$lookup?system=http://snomed.info/sct&code={0}&version=http://snomed.info/sct/83821000000107'.format(row[3])
                    params = requests.get(request,headers={"Accept":"application/fhir+json"})
                    #print(params.json())
                    jsonData = params.json()
                    if jsonData['resourceType'] == 'Parameters':
                        print('* $sct#{0} "{1}"'.format(row[3],jsonData['parameter'][4]['valueString']))
                except Exception as inst:
                    print(inst)
                    print('Missing {0}'.format(row[3]))



* $sct#163221000000102 "Allergies and Adverse Reactions - Care Record Element"
* $sct#927061000000101 "Gait analysis assessment record"
* $sct#149671000000105 "Health status questionnaire"
* $sct#734163000 "Care plan"
* $sct#820161000000108 "Growth chart"
* $sct#820191000000102 "Partogram"
* $sct#824231000000100 "Temperature chart"
* $sct#820211000000103 "Patient safety checklist"
* $sct#823611000000107 "Vital signs chart"
* $sct#820441000000103 "Weight chart"
* $sct#820221000000109 "Inpatient medical note"
* $sct#829201000000105 "Inpatient nursing note"
* $sct#820451000000100 "Medical note"
* $sct#820461000000102 "Multidisciplinary note"
* $sct#820471000000109 "Nursing note"
* $sct#823631000000104 "OOH (out of hours) note"
* $sct#820481000000106 "Outpatient nursing note"
* $sct#820491000000108 "Outpatient medical note"
* $sct#823641000000108 "AHP (allied health professional) note"
* $sct#24681000000104 "Telephone consultation"
* $sct#824321000000109 "Summary record"
* $sct#82365100000